<a href="https://colab.research.google.com/github/r42arty/hse/blob/main/term/red_remote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Бизнес-игра «Красная Удаленка»

## 🎯 Ваша цель — создать мультиагентную систему, способную эффективно управлять колонией на Марсе

# Технические настройки

In [ ]:
#@title Установка GigaChat в Jupiter
!pip install gigachat

In [ ]:
#@title Подключение библиотек
import pandas as pd
import random
import requests
import openai
from gigachat import GigaChat

In [ ]:
#@title Прописание токена GigaChat
TOKEN = "XXX"

### Вопрос для GigaChat

In [ ]:
QUESTION = 'Столица России?'

In [ ]:
#@title Обработка запроса
with GigaChat(credentials=TOKEN, verify_ssl_certs=False) as giga:
    Responce = giga.chat(QUESTION)
ANSWER = Responce.choices[0].message.content

### Ответ от GigaChat

In [ ]:
ANSWER

In [ ]:
#@title Базовые настройки игры

# Стартовые параметры
INITIAL_RESOURCES = 100
INITIAL_ENERGY = 10
INITIAL_SATISFACTION = {
    "Энергетики": 1/3,
    "Вычислители": 1/3,
    "Творцы": 1/3
}
CRITICAL_SATISFACTION = 1/5
MAX_DAYS = 30


# События
EVENT_POOL = [
    {"description": "Метеоритный дождь снижает выработку энергии", "energy_delta": -3, "resource_delta": 0, "faction_delta": {}},
    {"description": "Поставка оборудования с Земли", "energy_delta": 0, "resource_delta": 50, "faction_delta": {}},
    {"description": "Споры о приоритетах внутри колонии", "energy_delta": 0, "resource_delta": 0, "faction_delta": {"Творцы": -0.05, "Энергетики": -0.05}},
    {"description": "Успешный проект Творцов", "energy_delta": 0, "resource_delta": 0, "faction_delta": {"Творцы": 0.1}},
    {"description": "Отключение оборудования Вычислителей", "energy_delta": 0, "resource_delta": -20, "faction_delta": {"Вычислители": -0.1}},
]

# Функции симуляции
def apply_event(state, event):
    state['energy'] += event["energy_delta"]
    state['resources'] += event["resource_delta"]
    for faction, delta in event["faction_delta"].items():
        state["satisfaction"][faction] = max(0, min(1, state["satisfaction"][faction] + delta))
    return state

def run_simulation(mas_func, verbose=False):
    state = {
        "day": 1,
        "resources": INITIAL_RESOURCES,
        "energy": INITIAL_ENERGY,
        "satisfaction": INITIAL_SATISFACTION.copy(),
        "history": []
    }

    while state["day"] <= MAX_DAYS:
        if verbose:
            print(f"--- День {state['day']} ---")

        event = random.choice(EVENT_POOL)
        state = apply_event(state, event)

        decisions = mas_func(state)

        state['resources'] += decisions.get("resource_delta", 0)
        state['energy'] += decisions.get("energy_delta", 0)
        for faction, delta in decisions.get("faction_delta", {}).items():
            state["satisfaction"][faction] = max(0, min(1, state["satisfaction"][faction] + delta))

        state["history"].append({
            "day": state["day"],
            "event": event["description"],
            "state": {
                "resources": state["resources"],
                "energy": state["energy"],
                "satisfaction": state["satisfaction"].copy()
            }
        })

        if state["resources"] <= 0 or any(s < CRITICAL_SATISFACTION for s in state["satisfaction"].values()):
            break

        state["day"] += 1

    return state

# Начнем разработку вашей MAS👇

## 🌀 ФАЗА 1: Агенты и цели


In [ ]:
AGENTS_DESCRIPTION = """<Ваше описание агентов>"""

## 🌀 ФАЗА 2: Взаимодействие агентов

In [ ]:
INTERACTION_LOGIC = """<Ваше описание взаимодействия>"""

## 🌀 ФАЗА 3: Распределённая обработка

In [ ]:
DISTRIBUTED_DECISION = """<Ваше описание распределённого принятия решений>"""

## 🌀 ФАЗА 4: Адаптация и обучение

In [ ]:
ADAPTATION_MECHANISM = """<Ваше описание адаптации>"""

## 🌀 ФАЗА 5: Финальная настройка


In [ ]:
# Ниже напишите функцию mas_decision(initiative)
def mas_decision(initiative):
    # Пример: принимать всё, если хватает бюджета
    return state["budget"] >= initiative["cost"]

## 🌀 ФАЗА 6: Автономная симуляция

In [ ]:
# Загрузите CSV с инициативами от ведущего
# Файл должен содержать: faction,text,cost,satisfaction_change

csv_path = "initiatives_day6_to_30.csv"  # <-- замените на путь к своему файлу
future_events = pd.read_csv(csv_path)

# Преобразуем строку JSON в словарь satisfaction_change
future_events["satisfaction_change"] = future_events["satisfaction_change"].apply(ast.literal_eval)

# Симуляция автономной фазы
for i, row in future_events.iterrows():
    initiative = row.to_dict()
    decision = mas_decision(initiative)
    if decision:
        result = apply_initiative(state, initiative)
    else:
        result = "Отклонено"
    state["log"].append({"day": state["day"], "initiative": initiative, "decision": decision, "result": result})
    state["day"] += 1
    game_over, reason = check_game_over(state)
    if game_over:
        print(f"Игра завершена на дне {state['day'] - 1}: {reason}")
        break
else:
    print("MAS проработала все 30 дней!")